In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import pandas as pd
import warnings
import sys
import numpy as np
import scipy as sc
import random
from scipy import stats
from numpy import log10, sqrt

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


def Winsorize_it(x, WinZs):
    
    x2 = []
    for i, val in enumerate(x):
        wz = float(WinZs[i])
        val = float(val)
        if np.isnan(wz) == True:
            x2.append(np.nan)
        else:
            x2.append(val)
    
    p5 = np.nanpercentile(x2, 5)
    p95 = np.nanpercentile(x2, 95)
    WinScores = []
    
    for i, val in enumerate(x2):
        wz = float(WinZs[i])
        val = float(val)
        
        if np.isnan(wz) == True:
            WinScores.append(np.nan)
            
        elif np.isnan(wz) == False:
            if val >= p5 and val <= p95:
                WinScores.append(val)
            elif val < p5:
                WinScores.append(p5)
            elif val > p95:
                WinScores.append(p95)
            elif np.isnan(val) == True:
                WinScores.append(np.nan)
        
    
    return WinScores
        

def ZScore_it(x, WinZs):
    
    x2 = []
    for i, val in enumerate(x):
        wz = float(WinZs[i])
        val = float(val)
        if np.isnan(wz) == True:
            x2.append(np.nan)
        else:
            x2.append(val)
    
    x2 = np.array(x2)
    avg = np.nanmean(x2)
    std = np.nanstd(x2)
    zscores = (x2 - avg) / std
    return zscores

hac_mo = '10'

## Load files

In [2]:
main_df = pd.read_pickle('~/GitHub/HACRP-HAIs/data/Compiled_HCRIS-HACRP-HAI-RAND/Compiled_HCRIS-HACRP-HAI-RAND.pkl')
main_df = main_df[main_df['file_year'] == '2021']
main_df.sort_values(by='Facility ID', inplace=True)

print(main_df.shape[0], 'rows in hac_df')
print(len(main_df['Facility ID'].unique()), 'hospitals in hac_df')

##############  Label rows that have duplicate dates (per provider) ####################
##############  For each provider with rows having duplicate dates,  ###################

main_df.sort_values(by=['Facility ID', 'Line 19'], inplace=True, ascending=False)
main_df.drop_duplicates(subset=['Facility ID'], inplace=True, keep='first')

print(main_df.shape)

3204 rows in hac_df
3204 hospitals in hac_df
(3204, 100)


In [3]:
cauti_file =  '/Users/kenlocey/GitHub/HACRP-HAIs/4_optimize_random_sampling_models/optimized_by_HAI_file_date/CAUTI/CAUTI_Data_opt_for_SIRs_2021_10.pkl'
clabsi_file = '/Users/kenlocey/GitHub/HACRP-HAIs/4_optimize_random_sampling_models/optimized_by_HAI_file_date/CLABSI/CLABSI_Data_opt_for_SIRs_2021_10.pkl'
mrsa_file = '/Users/kenlocey/GitHub/HACRP-HAIs/4_optimize_random_sampling_models/optimized_by_HAI_file_date/MRSA/MRSA_Data_opt_for_SIRs_2021_10.pkl'
cdi_file = '/Users/kenlocey/GitHub/HACRP-HAIs/4_optimize_random_sampling_models/optimized_by_HAI_file_date/CDI/CDI_Data_opt_for_SIRs_2021_10.pkl'


In [4]:

tdf, tdf1, tdf2 = 0, 0, 0
cauti_df, clabsi_df = 0, 0
mrsa_df, cdi_df = 0, 0

cauti_df = pd.read_pickle(cauti_file)
cauti_df['CAUTI SIS'] = (cauti_df['CAUTI Observed Cases'] - cauti_df['expected O'])/cauti_df['CAUTI Predicted Cases']
cauti_df.rename(columns={
        'O/E': 'CAUTI O/E',
        'simulated O': 'CAUTI simulated O',
        'simulated O/E': 'CAUTI simulated O/E',
        'expected O': 'CAUTI expected O',
        'expected O/E': 'CAUTI expected O/E',
        'pi_opt': 'CAUTI pi_opt',
        'z_opt': 'CAUTI z_opt',
    }, inplace=True)
tdf = cauti_df[cauti_df['CAUTI Predicted Cases'] >= 1]
cauti_deciles = np.nanpercentile(tdf['CAUTI SIS'], np.arange(0, 100, 10))

    
clabsi_df = pd.read_pickle(clabsi_file)
clabsi_df['CLABSI SIS'] = (clabsi_df['CLABSI Observed Cases'] - clabsi_df['expected O'])/clabsi_df['CLABSI Predicted Cases']
clabsi_df.rename(columns={
        'O/E': 'CLABSI O/E',
        'simulated O': 'CLABSI simulated O',
        'simulated O/E': 'CLABSI simulated O/E',
        'expected O': 'CLABSI expected O',
        'expected O/E': 'CLABSI expected O/E',
        'pi_opt': 'CLABSI pi_opt',
        'z_opt': 'CLABSI z_opt',
    }, inplace=True)
tdf = clabsi_df[clabsi_df['CLABSI Predicted Cases'] >= 1]
clabsi_deciles = np.nanpercentile(tdf['CLABSI SIS'], np.arange(0, 100, 10))    
    
mrsa_df = pd.read_pickle(mrsa_file)
mrsa_df['MRSA SIS'] = (mrsa_df['MRSA Observed Cases'] - mrsa_df['expected O'])/mrsa_df['MRSA Predicted Cases']
mrsa_df.rename(columns={
        'O/E': 'MRSA O/E',
        'simulated O': 'MRSA simulated O',
        'simulated O/E': 'MRSA simulated O/E',
        'expected O': 'MRSA expected O',
        'expected O/E': 'MRSA expected O/E',
        'pi_opt': 'MRSA pi_opt',
        'z_opt': 'MRSA z_opt',
    }, inplace=True)
tdf = mrsa_df[mrsa_df['MRSA Predicted Cases'] >= 1]
mrsa_deciles = np.nanpercentile(tdf['MRSA SIS'], np.arange(0, 100, 10))    


cdi_df = pd.read_pickle(cdi_file)
cdi_df['CDI SIS'] = (cdi_df['CDI Observed Cases'] - cdi_df['expected O'])/cdi_df['CDI Predicted Cases']
cdi_df.rename(columns={
        'O/E': 'CDI O/E',
        'simulated O': 'CDI simulated O',
        'simulated O/E': 'CDI simulated O/E',
        'expected O': 'CDI expected O',
        'expected O/E': 'CDI expected O/E',
        'pi_opt': 'CDI pi_opt',
        'z_opt': 'CDI z_opt',
    }, inplace=True)
tdf = cdi_df[cdi_df['CDI Predicted Cases'] >= 1]
cdi_deciles = np.nanpercentile(tdf['CDI SIS'], np.arange(0, 100, 10))    


In [5]:
opt_df = cauti_df.copy(deep=True)
ls1 = list(opt_df)
ls2 = list(clabsi_df)
ls = list(filter(lambda x:x in ls1, ls2))
opt_df = opt_df.merge(clabsi_df, on=ls, how='outer')
    
ls1 = list(opt_df)
ls2 = list(mrsa_df)
ls = list(filter(lambda x:x in ls1, ls2))
opt_df = opt_df.merge(mrsa_df, on=ls, how='outer')

ls1 = list(opt_df)
ls2 = list(cdi_df)
ls = list(filter(lambda x:x in ls1, ls2))
opt_df = opt_df.merge(cdi_df, on=ls, how='outer')


del tdf, tdf1, tdf2
del cauti_df, clabsi_df
del mrsa_df, cdi_df

opt_df.sort_values(by=['Facility ID', 'Line 19'], inplace=True, ascending=False)
opt_df.drop_duplicates(subset=['Facility ID'], inplace=True, keep='first')

opt_df.head()


,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final",CAUTI O/E,CAUTI simulated O,CAUTI simulated O/E,CAUTI expected O,CAUTI expected O/E,CAUTI pi_opt,CAUTI z_opt,CAUTI SIS,CLABSI O/E,CLABSI simulated O,CLABSI simulated O/E,CLABSI expected O,CLABSI expected O/E,CLABSI pi_opt,CLABSI z_opt,CLABSI SIS,MRSA O/E,MRSA simulated O,MRSA simulated O/E,MRSA expected O,MRSA expected O/E,MRSA pi_opt,MRSA z_opt,MRSA SIS,CDI O/E,CDI simulated O,CDI simulated O/E,CDI expected O,CDI expected O/E,CDI pi_opt,CDI z_opt,CDI SIS
0,720071,47355055.0,51684587.0,2,670122,2021,51532086.0,51500482.0,515005.0,50985477.0,516845.87,51684576.0,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31,NaN,2018-01-01,HOUSTON METHODIST THE WOODLANDS HOSPITAL,2021.0,TX,-1.1071,10,No,16702.0,9533.0,7169.0,3.0,0.0,6.510,5.220,0.4608,0.0000,NaN,NaN,-1.107365,No,1.0,0.000265,-0.001676,0.000549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67044.0,63442.0,1.0,10.0,2.773,43.473,0.3606,0.2300,NaN,NaN,0.002373,5.447727e-04,NaN,-0.5867,-1.2042,-1.3958,NaN,NaN,-0.8146,NaN,-1.1556,-1.4855,0.4608,-0.585024,0.000000,-1.396349,0.3606,-0.815145,0.0,0.2300,-1.206573,06/30/2019,07/01/2017,516846.0,2021_10,TX,11.0,0.000021,NaN,NaN,NaN,NaN,451290.666667,515005.00,0.460829,4.0,0.614439,6.232323,0.957346,0.001365,10366.162678,-0.496517,0.000000,7.0,1.340996,3.546811,0.679466,0.000779,4120.152382,-0.679466,0.36062,3.0,1.081861,2.199404,0.793150,0.00008,97126.622645,-0.432529,0.230028,20.0,0.460056,22.395443,0.515158,0.000453,18055.42641,-0.285130
1,720752,13387779.0,13967410.0,4,670120,2021,13543672.0,13473517.0,134735.0,13338782.0,139674.10,13967408.0,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31,NaN,2018-01-01,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2021.0,TX,-0.6383,10,No,5786.0,3461.0,2325.0,0.0,0.0,2.178,1.659,0.0000,0.0000,NaN,NaN,-0.637740,No,1.0,-0.000560,-0.003836,0.000549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [6]:
ls1 = list(main_df)
ls2 = list(opt_df)
ls = list(filter(lambda x:x in ls1, ls2))
main_df = main_df.merge(opt_df, on=ls, how='outer')

print(main_df.shape)
main_df.head()

(3204, 132)


,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final",CAUTI O/E,CAUTI simulated O,CAUTI simulated O/E,CAUTI expected O,CAUTI expected O/E,CAUTI pi_opt,CAUTI z_opt,CAUTI SIS,CLABSI O/E,CLABSI simulated O,CLABSI simulated O/E,CLABSI expected O,CLABSI expected O/E,CLABSI pi_opt,CLABSI z_opt,CLABSI SIS,MRSA O/E,MRSA simulated O,MRSA simulated O/E,MRSA expected O,MRSA expected O/E,MRSA pi_opt,MRSA z_opt,MRSA SIS,CDI O/E,CDI simulated O,CDI simulated O/E,CDI expected O,CDI expected O/E,CDI pi_opt,CDI z_opt,CDI SIS
0,NaN,NaN,NaN,NaN,670143,2021,0.0,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31,NaN,2018-01-01,ASCENSION SETON BASTROP,2021.0,TX,NaN,10,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,NaN,5.0,5.0,NaN,5.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,06/30/2019,07/01/2017,NaN,2021_10,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,722480,1087995.0,2109840.0,2,670136,2021,2109840.0,2109840.0,21098.0,2088742.0,21098.40,2109840.0,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31,NaN,2018-01-01,BAYLOR SCOTT & WHITE MEDICAL CENTER- AUSTIN,2021.0,TX,NaN,10,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,NaN,5.0,5.0,NaN,5.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,06/30/2019,07/01/2017,21098.0,2021_10,TX,0.0,0.0,NaN,NaN,NaN,NaN,21098.0,21098.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,724900,173976.0,193134.0,5,670135,2021,193134.0,193134.0,1931.0,191203.0,1931.34,193134.0,NaN,NaN,NaN,Na

In [7]:
print(main_df['file_year'].unique())
hac_df = main_df[main_df['file_year'] == '2021']
hac_df = main_df.copy(deep=True)
hac_df = hac_df[hac_df['file_month'] == hac_mo]
hac_df.dropna(how='all', axis=1, inplace=True)
hac_df.head()

['2021']


,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),End Date,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,CAUTI Footnote,CLABSI Footnote,SSI Footnote,Total HAC Footnote,CDI Footnote,MRSA Footnote,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final",CAUTI O/E,CAUTI simulated O,CAUTI simulated O/E,CAUTI expected O,CAUTI expected O/E,CAUTI pi_opt,CAUTI z_opt,CAUTI SIS,CLABSI O/E,CLABSI simulated O,CLABSI simulated O/E,CLABSI expected O,CLABSI expected O/E,CLABSI pi_opt,CLABSI z_opt,CLABSI SIS,MRSA O/E,MRSA simulated O,MRSA simulated O/E,MRSA expected O,MRSA expected O/E,MRSA pi_opt,MRSA z_opt,MRSA SIS,CDI O/E,CDI simulated O,CDI simulated O/E,CDI expected O,CDI expected O/E,CDI pi_opt,CDI z_opt,CDI SIS
0,NaN,NaN,NaN,NaN,670143,2021,0.0,0.0,0.0,0.0,0.00,NaN,2019-12-31,2018-01-01,ASCENSION SETON BASTROP,2021.0,TX,NaN,10,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,06/30/2019,07/01/2017,NaN,2021_10,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,722480,1087995.0,2109840.0,2,670136,2021,2109840.0,2109840.0,21098.0,2088742.0,21098.40,2109840.0,2019-12-31,2018-01-01,BAYLOR SCOTT & WHITE MEDICAL CENTER- AUSTIN,2021.0,TX,NaN,10,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,06/30/2019,07/01/2017,21098.0,2021_10,TX,0.0,0.0,NaN,NaN,NaN,NaN,21098.0,21098.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,724900,173976.0,193134.0,5,670135,2021,193134.0,193134.0,1931.0,191203.0,1931.34,193134.0,2019-12-31,2018-01-01,"ALTUS HOUSTON HOSPITAL, LP",2021.0,TX,NaN,10,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,06/30/2019,07/01/2017,1931.0,2021_10,TX,0.0,0.0,NaN,NaN,NaN,NaN,1931.0,1931.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,722600,218396.0,2747

## Attempt to reproduce HAC scores for 2021

In [8]:
df_yr = hac_df[hac_df['file_year'] == '2021']
df_yr.head()

,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),End Date,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,CAUTI Footnote,CLABSI Footnote,SSI Footnote,Total HAC Footnote,CDI Footnote,MRSA Footnote,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final",CAUTI O/E,CAUTI simulated O,CAUTI simulated O/E,CAUTI expected O,CAUTI expected O/E,CAUTI pi_opt,CAUTI z_opt,CAUTI SIS,CLABSI O/E,CLABSI simulated O,CLABSI simulated O/E,CLABSI expected O,CLABSI expected O/E,CLABSI pi_opt,CLABSI z_opt,CLABSI SIS,MRSA O/E,MRSA simulated O,MRSA simulated O/E,MRSA expected O,MRSA expected O/E,MRSA pi_opt,MRSA z_opt,MRSA SIS,CDI O/E,CDI simulated O,CDI simulated O/E,CDI expected O,CDI expected O/E,CDI pi_opt,CDI z_opt,CDI SIS
0,NaN,NaN,NaN,NaN,670143,2021,0.0,0.0,0.0,0.0,0.00,NaN,2019-12-31,2018-01-01,ASCENSION SETON BASTROP,2021.0,TX,NaN,10,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,06/30/2019,07/01/2017,NaN,2021_10,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,722480,1087995.0,2109840.0,2,670136,2021,2109840.0,2109840.0,21098.0,2088742.0,21098.40,2109840.0,2019-12-31,2018-01-01,BAYLOR SCOTT & WHITE MEDICAL CENTER- AUSTIN,2021.0,TX,NaN,10,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,06/30/2019,07/01/2017,21098.0,2021_10,TX,0.0,0.0,NaN,NaN,NaN,NaN,21098.0,21098.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,724900,173976.0,193134.0,5,670135,2021,193134.0,193134.0,1931.0,191203.0,1931.34,193134.0,2019-12-31,2018-01-01,"ALTUS HOUSTON HOSPITAL, LP",2021.0,TX,NaN,10,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,06/30/2019,07/01/2017,1931.0,2021_10,TX,0.0,0.0,NaN,NaN,NaN,NaN,1931.0,1931.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,722600,218396.0,2747

## Generate Winsorized z-scores

In [9]:
hais = ['CAUTI', 'CLABSI', 'MRSA', 'CDI']
    
for i, hai in enumerate(hais):
    tdf2 = df_yr[~df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18', 
                                              5, '5', ' 5', '5 ',
                                              4, '4', ' 4', '4 ',
                                              ])]
      
    reported_winZ = tdf2[hai + ' W Z Score'].tolist()
    sirs = tdf2[hai + ' SIS'].tolist()
    tdf2[hai + ' Winsorized SIS'] = Winsorize_it(sirs, reported_winZ)
    tdf2[hai + ' SIS W Z Score'] = ZScore_it(tdf2[hai + ' Winsorized SIS'], reported_winZ)
    
    # Assign maximum WinZ scores to hospitals with HAI footnote 18 
    maxWinZ = np.nanmax(tdf2[hai + ' SIS W Z Score'])
    tdf3 = df_yr[df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    tdf3[hai + ' Winsorized SIS'] = [np.nan]*tdf3.shape[0]
    tdf3[hai + ' SIS W Z Score'] = [maxWinZ]*tdf3.shape[0]
    
    tdf4 = df_yr[df_yr[hai + ' Footnote'].isin([5, '5', ' 5', '5 ', 4, '4', ' 4', '4 '])]
    tdf4[hai + ' Winsorized SIS'] = [np.nan]*tdf4.shape[0]
    tdf4[hai + ' SIS W Z Score'] = [np.nan]*tdf4.shape[0]
    
    df_yr = pd.concat([tdf2, tdf3, tdf4], axis=0)

del tdf2
del tdf3

## Attempt to reproduce HAC scores for 2020

In [10]:
print('Results from attempting to reproduce Yes/No penalty assignments:\n')
print('Excluded from results below:')
print('1. MD hospitals')
print('2. Hospitals with payment reduction values other than Yes or No\n')

holdout_df = df_yr[(df_yr['State'] == 'MD') | ~df_yr['Payment Reduction'].isin(['Yes', 'No']) | (df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]
df_yr = df_yr[(df_yr['State'] != 'MD') & (df_yr['Payment Reduction'].isin(['Yes', 'No'])) & (~df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]

hac_scores = []
ct1 = 0
ct2 = 0


for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    d = 0
        
    w_ls = []
    sum_ls = []

    m_ls = ['PSI-90 W Z Score', 'CDI SIS W Z Score', 'CAUTI SIS W Z Score', 'CLABSI SIS W Z Score', 'MRSA SIS W Z Score', 'SSI W Z Score']
                
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d = np.nan
    else:
        d = s/w
        
    # If the derived score is NaN and the original score is NaN ...
    if np.isnan(d) == True:
        hac_scores.append(np.nan)
        
    # If the derived score is a float and the original score is a float ...
    elif np.isnan(d) == False:
        hac_scores.append(d)

print('holdout_df.shape:', holdout_df.shape)
df_yr['Total HAC Score (SIS-based)'] = hac_scores
print(df_yr.shape[0], 'hospitals in hac_df')


Results from attempting to reproduce Yes/No penalty assignments:

Excluded from results below:
1. MD hospitals
2. Hospitals with payment reduction values other than Yes or No

holdout_df.shape: (99, 121)
3105 hospitals in hac_df


In [11]:
p75 = np.nanpercentile(df_yr['Total HAC Score (SIS-based)'], 75)
print('p75:', p75)

pr = []
for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    score = tdf['Total HAC Score (SIS-based)'].iloc[0]

    if np.isnan(score) == True:
        pr.append('No')
    elif score <= p75:
        pr.append('No')
    elif score > p75:
        pr.append('Yes')
    else:
        print('This score is an error:', score)
        sys.exit()

df_yr['Payment Reduction (SIS-based)'] = pr
    
o_list = df_yr['Payment Reduction'].tolist()
d_list = df_yr['Payment Reduction (SIS-based)'].tolist()

same = 0
diff = 0
p_to_np = 0
np_to_p = 0

res_ls = []
for i, o in enumerate(o_list):
    if o == d_list[i]:
        same += 1
        res_ls.append(1)
    else:
        diff += 1
        if o == 'Yes' and d_list[i] == 'No':
            p_to_np += 1
        elif o == 'No' and d_list[i] == 'Yes':
            np_to_p += 1
        else:
            print('Error')
            break
        res_ls.append(0)
               
print(same, "SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments.")
print(diff, "SIR-based penalty assignments were NOT the same as their corresponding SIS-based penalty assignments.")
print(str(np.round(100 * same/(same+diff),2)) + '% of SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments."\n')

print(p_to_np, 'hospitals were penalized but should not have been.')
print(np_to_p, 'hospitals were NOT penalized but should have been.')

p75: 0.33285668173414124
2863 SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments.
242 SIR-based penalty assignments were NOT the same as their corresponding SIS-based penalty assignments.
92.21% of SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments."

120 hospitals were penalized but should not have been.
122 hospitals were NOT penalized but should have been.


In [12]:
tdf1 = df_yr[df_yr['Payment Reduction (SIS-based)'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction (SIS-based)'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction (SIS-based)'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])


0.2499194847020934
0.7500805152979066


In [13]:
tdf1 = df_yr[df_yr['Payment Reduction'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])


0.2492753623188406
0.7507246376811594


In [14]:
ls1 = list(df_yr)
ls2 = list(holdout_df)
ls = list(filter(lambda x:x in ls1, ls2))
print(df_yr.shape)
print(holdout_df.shape)
df_yr = df_yr.merge(holdout_df, how='outer', on=ls)
print(df_yr.shape)

(3105, 123)
(99, 121)
(3204, 123)


In [15]:
p_np_df = df_yr[(df_yr['Payment Reduction'] == 'Yes') & (df_yr['Payment Reduction (SIS-based)'] == 'No')]
ip_ = np.round(np.nansum(p_np_df['HAC penalty, final']))
print(p_np_df.shape[0])

np_p_df = df_yr[(df_yr['Payment Reduction'] == 'No') & (df_yr['Payment Reduction (SIS-based)'] == 'Yes')]
is_ = np.round(np.nansum(np_p_df['HAC penalty, final']))
print(np_p_df.shape[0])

print(ip_, 'dollars of inappropriate penalties')
print(is_, 'dollars of inappropriate hospital savings')
print('', ip_ - is_)


120
122
79761350.0 dollars of inappropriate penalties
21771812.0 dollars of inappropriate hospital savings
 57989538.0


In [16]:
df_yr.to_pickle('~/GitHub/HACRP-HAIs/data/finalized/final_2021.pkl', protocol=5)